In [69]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import json
import requests
from io import BytesIO
from pydub import AudioSegment
from tqdm import tqdm
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datasets/riyajolly/dataset/FT Data - data.csv
/kaggle/input/datasets/riyajolly/dist-df/Speech Disfluencies List - Sheet1.csv


In [52]:
df=pd.read_csv("/kaggle/input/datasets/riyajolly/dataset/FT Data - data.csv")

In [53]:
df.head()

,user_id,recording_id,language,duration,rec_url_gcp,transcription_url_gcp,metadata_url_gcp
0,245746,825780,hi,443,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...
1,291038,825727,hi,443,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...
2,246004,988596,hi,475,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...
3,93626,990175,hi,475,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...
4,286851,526266,hi,522,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...


In [54]:
dist_df=pd.read_csv("/kaggle/input/datasets/riyajolly/dist-df/Speech Disfluencies List - Sheet1.csv")

In [55]:
dist_df.head()

,Filled Pause,Repetition,False Start,Prolongation,Self-Correction
0,अं,मैं-मैं,जा—,अच्छ्छ्छा,कल—
1,अँ,वो-वो,कर—,हम्म्म,नहीं—
2,उम्,ये-ये,ले—,आाा,परसों—
3,NaN,जी-जी,कह—,अरे रे रे,माफ़—
4,हम्,उह उह,वो तो,अह,नहीं तो हां तो ये


In [56]:
disfluency_list=[]
for col in dist_df.columns:
    disfluency_list.extend(dist_df[col].dropna().astype(str).tolist())

disfluency_list=list(set(disfluency_list))
print("Total disfluency patterns: ",len(disfluency_list))

Total disfluency patterns:  193


In [57]:
def contains_disfluency(text):
    text=str(text)
    for pattern in disfluency_list:
        if pattern in text:
            return pattern

    return None

In [58]:
os.makedirs("/kaggle/working/disfluency_clips", exist_ok=True)


In [59]:
#Fixing broken urls
df["trans_url_fixed"]=df["transcription_url_gcp"].str.replace("joshtalks-data-collection/hq_data/hi/", "upload_goai/", regex=False)
df["audio_url_fixed"]=df["rec_url_gcp"].str.replace("joshtalks-data-collection/hq_data/hi/", "upload_goai/", regex=False)

In [60]:
df

,user_id,recording_id,language,duration,rec_url_gcp,transcription_url_gcp,metadata_url_gcp,trans_url_fixed,audio_url_fixed
0,245746,825780,hi,443,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/967...,https://storage.googleapis.com/upload_goai/967...
1,291038,825727,hi,443,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/967...,https://storage.googleapis.com/upload_goai/967...
2,246004,988596,hi,475,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/114...,https://storage.googleapis.com/upload_goai/114...
3,93626,990175,hi,475,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/114...,https://storage.googleapis.com/upload_goai/114...
4,286851,526266,hi,522,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/639...,https://storage.googleapis.com/upload_goai/639...
...,...,...,...,...,...,...,...,...,...
99,278010,753435,hi,589,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/887...,https://storage.googleapis.com/upload_goai/887...
100,413240,1021370,hi,1194,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/118...,https://storage.googleapis.com/upload_goai/118...
101,11057,1020918,hi,1194,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/118...,https://storage.googleapis.com/upload_goai/118...
102,93299,840793,hi,1146,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/983...,https://storage.googleapis.com/upload_goai/983...


In [61]:
#dropping the columns that are not required
df.drop(["user_id",	"language",	"duration", "transcription_url_gcp", "metadata_url_gcp"],axis=1, inplace=True)

In [62]:
df

,recording_id,rec_url_gcp,trans_url_fixed,audio_url_fixed
0,825780,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/967...,https://storage.googleapis.com/upload_goai/967...
1,825727,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/967...,https://storage.googleapis.com/upload_goai/967...
2,988596,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/114...,https://storage.googleapis.com/upload_goai/114...
3,990175,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/114...,https://storage.googleapis.com/upload_goai/114...
4,526266,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/639...,https://storage.googleapis.com/upload_goai/639...
...,...,...,...,...
99,753435,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/887...,https://storage.googleapis.com/upload_goai/887...
100,1021370,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/118...,https://storage.googleapis.com/upload_goai/118...
101,1020918,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/118...,https://storage.googleapis.com/upload_goai/118...
102,840793,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/983...,https://storage.googleapis.com/upload_goai/983...


In [63]:
df["audio_url_fixed"][0]

'https://storage.googleapis.com/upload_goai/967179/825780_audio.wav'

In [83]:
os.makedirs('segmented_clips', exist_ok=True)

results = []

def process_recordings():

    for index, row in df.head().iterrows():

        rec_id = row['recording_id']
        audio_url = row['audio_url_fixed']
        trans_url = row['trans_url_fixed']

        print(f"Processing Recording ID: {rec_id}")

        try:
            trans_resp = requests.get(trans_url)
            trans_data = trans_resp.json()

            audio_resp = requests.get(audio_url)
            audio = AudioSegment.from_file(BytesIO(audio_resp.content))

            # Analyze each segment
            for segment in trans_data:

                # ⭐ Correct timestamp keys
                start = segment.get('start')
                end = segment.get('end')
                text = segment.get('text', '')

                # ⭐ Skip invalid timestamps
                if start is None or end is None:
                    continue

                start_ms = int(start * 1000)
                end_ms = int(end * 1000)

                # Detect disfluency
                found_disfluency = None
                for d in disfluency_list:
                    if d in text:
                        found_disfluency = d
                        break

                if found_disfluency:
                    # Clip audio
                    clip = audio[start_ms:end_ms]

                    clip_filename = f"{rec_id}_{start_ms}.wav"
                    clip_path = os.path.join('segmented_clips', clip_filename)

                    clip.export(clip_path, format="wav")

                    # Record dataset entry
                    results.append({
                        'recording_id': rec_id,
                        'disfluency_type': found_disfluency,
                        'audio_segment_url': clip_path,
                        'start_time': start,
                        'end_time': end,
                        'transcription_segment': text
                    })

        except Exception as e:
            print(f"Error processing {rec_id}: {e}")

# Run processing
process_recordings()

# Export final dataset
output_sheet = pd.DataFrame(results)
output_sheet.to_csv('disfluency_dataset.csv', index=False)

print("Dataset Created: disfluency_dataset.csv")

Processing Recording ID: 825780
Processing Recording ID: 825727
Processing Recording ID: 988596
Processing Recording ID: 990175
Processing Recording ID: 526266
Dataset Created: disfluency_dataset.csv


In [81]:
disfluency_dataset=pd.read_csv("disfluency_dataset.csv")

In [84]:
disfluency_dataset

,recording_id,disfluency_type,transcription_segment,start_time,end_time,audio_segment_url
0,825780,वो तो,अब काफी अच्छा होता है क्योंकि उनकी जनसंख्या बह...,0.11,14.42,segmented_clips/825780_110.wav
1,825780,हम्,हम्म हम लोग की तो सुबह होती है की मम्मी उठाने ...,130.25,144.83,segmented_clips/825780_130250.wav
2,825780,अं,बहुत अजीब सा अनुभव था क्योंकि वो पता है उठा उठ...,160.67,173.24,segmented_clips/825780_160670.wav
3,825727,हा हा,"हा हा,पहली बार था, पहली बार था,क्या आप लोग का ...",39.83,53.78,segmented_clips/825727_39830.wav
4,825727,बिल,बिल्कुल।,59.21,59.72,segmented_clips/825727_59210.wav
5,825727,आह,जी जी जी आह,101.36,104.18,segmented_clips/825727_101360.wav
6,825727,हा हा,ही बहुत जी जी हा हा,120.20,126.14,segmented_clips/825727_120200.wav
7,825727,अच्छ,हा अच्छा अच्छा मतलब एक मतलब अजीब साउन बहुत हाँ...,155.42,162.05,segmented_clips/825727_155420.wav
8,825727,अह,हाहाहाआपको डर भी लग रहा होगा और अजीब ही अहम हु...,166.28,173.12,segmented_clips/825727_166280.wav
9,825727,अच्छ,अच्छा अच्छा अह जहां तक मेरा एक्सपीरियंस रहा है...,199.10,214.01,segmented_clips/825727_199100.wav


# Methodology

### How you detected the disfluencies?
A pre-defined list of Hindi disfluency patterns were provided as a part of the assignment. Each transcription section was scanned and if any of the text matched the items of the list, then that segment was marked as containing a disfluency.

### How you clipped the audio segment from the complete recording?
The audio segments were clipped using the start and end timestamps provided in the transcription metadata JSON files. The full recording was downloaded and then pydub audio processing library was used to extract the respective portion of the audio based on timestamps. each clip was saved as a separate WAV file.

### Any preprocessing/normalization applied?
In the original dataset, the links were broken. They were first fixed. Basic preprocessing like dropping null values, removing duplicate disfluency patterns from the disfluency list. Audio recording was normalized by converting timestamps from seconds to milliseconds.